# Generating recordings with varying noise level

This notebook shows how to use MEArec to create recordings with different levels of noise. 
The same principle can be used to study the effect of other features/parameters (e.g. bursting, drifting) on spike sorting.

The generated recodrings will share the same spiking activity and only change in the noise level. Before running this notebook, you need to have generated some templates with the 
`generate_templates_and_recordings.ipynb` notebook.

In [ ]:
import MEArec as mr
import numpy as np
import matplotlib.pylab as plt
from pprint import pprint
import yaml 
%matplotlib notebook

Let's first take a look at the available parameters to generate recordings:

In [ ]:
rec_params  =mr.get_default_recordings_params()
pprint(rec_params)

We can now change some parameters *globally* and load the templates. 
It is important to notice that setting the `spiketrain` and the `templates` seeds ensures that the same spiking activity is replicated (the same random spike trains will be generated, and the same templates will be selected).

In [ ]:
# change parameters
rec_params['spiketrains']['n_exc'] = 7
rec_params['spiketrains']['n_inh'] = 3
rec_params['templates']['max_amp'] = 300
rec_params['recordings']['duration'] = 20

# set seeds
rec_params['spiketrains']['seed'] = np.random.randint(1000)
rec_params['templates']['seed'] = np.random.randint(1000)

# select templates
template_file = 'data/test_templates.h5'

rec_params['recordings']['drifting'] = False
rec_params['recordings']['noise_mode'] = 'uncorrelated'


Now let's defin a list of noise levels and simulate many recordings by sweeping through the noise levels. We can save the list of `RecordingGenerator` objects in the `recgen_list` list:

In [ ]:
noise_levels = [5, 10, 15, 20, 25, 30, 50]

recgen_list = []

for n in noise_levels:
    print('Noise level: ', n)
    rec_params['recordings']['noise_level'] = n
    rec_params['recordings']['seed'] = np.random.randint(1000)
    recgen = mr.gen_recordings(templates=template_file, params=rec_params, verbose=False)
    recgen_list.append(recgen)

Let's now plot the recordings! We'll reverse the noise order so that recordings with large noise levels sit at the bottom.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, recgen in enumerate(recgen_list[::-1]):
    mr.plot_recordings(recgen, ax=ax, end_time=1, colors=colors[i])